In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.utils import class_weight

2023-05-04 15:21:41.849492: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 15:21:42.783574: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv("../../data/flags/flags.csv", header=None)

flags = set([(df[1][i],df[2][i],df[3][i]) for i in range(0,14)])

colors = ["red","lightblue","blue","green","orange","yellow","white"]

x = []
y = []

for c1 in colors:
    x.append(c1)
    if c1 in [x[0] for x in flags]:
        y.append(1)
    else:
        y.append(0)
    
    for c2 in colors:
        x.append(c1+","+c2)
        if c2 in [x[1] for x in flags if x[0]==c1]:
            y.append(1)
        else:
            y.append(0)
        
        for c3 in colors:
            x.append(c1+","+c2+","+c3)
            if c3 in [x[2] for x in flags if x[0]==c1 and x[1]==c2]:
                y.append(1)
            else:
                y.append(0)

x = np.array(x)

y = np.array(y)

In [3]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=20, split=",")

# updates internal vocabulary based on the lyrics
tokenizer.fit_on_texts(x)

# transform each text in x to a sequence of tokens
x = tokenizer.texts_to_sequences(x)

x = np.array(pad_sequences(x, maxlen = 3))

print(x)

[[0 0 1]
 [0 1 1]
 [1 1 1]
 ...
 [7 7 5]
 [7 7 6]
 [7 7 7]]


In [4]:
class_weights = class_weight.compute_class_weight('balanced', classes = np.unique(y), y=y)

In [5]:
num = len(y)

idx = np.random.permutation(num)

x = x[idx]
y = y[idx]

x = x.reshape((x.shape[0], x.shape[1], 1))

n_classes = len(np.unique(y))

np.unique(y)

print(n_classes)

2


In [6]:

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [7]:

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inputs, outputs)


In [8]:
print(class_weights)

import keras.backend as K

def weighted_binary_crossentropy(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)
    # Flatten the predictions and true labels to compute the cross-entropy loss
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    # Compute the binary cross-entropy loss
    bce = K.binary_crossentropy(y_true_f, y_pred_f)
    # Multiply the loss by the class weights
    weight_vector = y_true_f * class_weights[1] + (1. - y_true_f) * class_weights[0]
    weighted_bce = weight_vector * bce
    return K.mean(weighted_bce)

[0.54065041 6.65      ]


In [9]:
input_shape = x.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[256],
    mlp_dropout=0,
    dropout=0,
)

model.compile(
    loss=weighted_binary_crossentropy,
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=["accuracy"]
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x,
    y,
    validation_data=(x,y),
    #validation_split=0.2,
    epochs=500,
    batch_size=32,
    callbacks=callbacks,
)

model.evaluate(x, y, verbose=1)

2023-05-04 15:21:45.287953: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-04 15:21:45.329657: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-04 15:21:45.329995: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3, 1)]       0           []                               
                                                                                                  
 layer_normalization (LayerNorm  (None, 3, 1)        2           ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 multi_head_attention (MultiHea  (None, 3, 1)        7169        ['layer_normalization[0][0]',    
 dAttention)                                                      'layer_normalization[0][0]']    
                                                                                              

2023-05-04 15:21:54.362740: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-05-04 15:21:55.151335: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fa6ec01d430 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-04 15:21:55.151363: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce GTX 960M, Compute Capability 5.0
2023-05-04 15:21:55.159690: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-04 15:21:55.341951: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


13/13 [==============================] - 18s 87ms/step - loss: 0.6570 - accuracy: 0.8922 - val_loss: 0.5985 - val_accuracy: 0.7318
Epoch 2/500
13/13 [==============================] - 0s 27ms/step - loss: 0.5801 - accuracy: 0.7018 - val_loss: 0.5515 - val_accuracy: 0.6792
Epoch 3/500
13/13 [==============================] - 0s 27ms/step - loss: 0.5472 - accuracy: 0.6792 - val_loss: 0.5262 - val_accuracy: 0.6917
Epoch 4/500
13/13 [==============================] - 0s 27ms/step - loss: 0.5447 - accuracy: 0.5965 - val_loss: 0.5159 - val_accuracy: 0.6291
Epoch 5/500
13/13 [==============================] - 0s 27ms/step - loss: 0.5126 - accuracy: 0.7318 - val_loss: 0.5089 - val_accuracy: 0.7744
Epoch 6/500
13/13 [==============================] - 0s 28ms/step - loss: 0.5028 - accuracy: 0.7519 - val_loss: 0.4916 - val_accuracy: 0.6792
Epoch 7/500
13/13 [==============================] - 0s 28ms/step - loss: 0.4944 - accuracy: 0.6942 - val_loss: 0.4852 - val_accuracy: 0.6717
Epoch 8/500
13/13

[0.21324266493320465, 0.8872180581092834]

In [10]:
model.save('../models/flags_validator_model')

# save tokenizer to file
f = open("../models/flags_validator_model/tokenizer.json", "w")
f.write(tokenizer.to_json())
f.close()

INFO:tensorflow:Assets written to: ../models/flags_validator_model/assets


INFO:tensorflow:Assets written to: ../models/flags_validator_model/assets
